In [1]:
import pandas as pd
import json

# Create top 10 per country 

In [2]:
df = pd.read_pickle('data/rankings.pkl')

In [3]:
df.date = pd.to_datetime(df.date)
df['year'] = df['date'].dt.year


In [4]:
df.head()

,artist_id,country,date,chartType,artist_name,image_link,currentRank,previousRank,peakRank,peakDate,appearancesOnChart,consecutiveAppearancesOnChart,entryStatus,entryRank,entryDate,year
0,4q3ewBCX7sLwd24euuV69X,GLOBAL,2025-05-15,TOP_ARTIST,Bad Bunny,https://i.scdn.co/image/ab6761610000517481f47f...,1,1,1,2021-10-28,187,187,NO_CHANGE,2,2021-10-21,2025
1,06HL4z0CvFAxyc27GXpf02,GLOBAL,2025-05-15,TOP_ARTIST,Taylor Swift,https://i.scdn.co/image/ab67616100005174e672b5...,2,2,1,2021-11-11,187,187,NO_CHANGE,6,2021-10-21,2025
2,1Xyo4u8uXC1ZmMpatF05PJ,GLOBAL,2025-05-15,TOP_ARTIST,The Weeknd,https://i.scdn.co/image/ab676161000051749e5289...,3,4,1,2022-01-13,187,187,MOVED_UP,11,2021-10-21,2025
3,3TVXtAsR1Inumwj472S9r4,GLOBAL,2025-05-15,TOP_ARTIST,Drake,https://i.scdn.co/image/ab67616100005174429338...,4,3,1,2023-10-12,187,187,MOVED_DOWN,3,2021-10-21,2025
4,0ys2OFYzWYB5hRDLCsBqxt,GLOBAL,2025-05-15,TOP_ARTIST,Fuerza Regida,https://i.scdn.co/image/ab67616100005174158a3d...,5,7,5,2025-05-15,128,128,MOVED_UP,190,2022-12-08,2025


In [19]:
def compute_ranking(artistDf):
    rank = artistDf.currentRank.sum()
    return rank

In [32]:
result = {}


grouped = df.groupby(['year', 'country'])

for (year, country, ), group in grouped:
    byArtist = group.groupby('artist_name')
    year = int(year)

    ranksToArtist = {}
    artistsToRank = {}
    artistsRow = {}

    for artist, artistDf in byArtist:
        rank = compute_ranking(artistDf)

        if rank in ranksToArtist:
            ranksToArtist[rank].append(artist)
        else:   
            ranksToArtist[rank] = [artist]

        artistsToRank[artist] = int(rank)
        artistsRow[artist] = artistDf.iloc[0]

    ranksSorted = sorted(ranksToArtist.keys())

    sortedArtists = []
    for rank in ranksSorted[:10]:
        if len(sortedArtists) >= 10:
            break
        artists = ranksToArtist[rank]
        for artist in artists:
            if len(sortedArtists) >= 10:
                break
            sortedArtists.append(artist)

    if year not in result:
        result[year] = {}
    if country not in result[year]:
        result[year][country] = {}
    for i, artist in enumerate(sortedArtists):
        rank_key = int(i+1)
        if rank_key not in result[year][country]:
            result[year][country][rank_key] = {}
        result[year][country][rank_key]['artist'] = artist
        result[year][country][rank_key]['aggregated_rank'] = artistsToRank[artist]
        if pd.notna(artistsRow[artist].image_link):
            result[year][country][rank_key]['image'] = artistsRow[artist].image_link
        else :
            result[year][country][rank_key]['image'] = None


with open('top10_artist_by_country.json', 'w') as f:
    json.dump(result, f, indent=2, )